In [3]:
import pandas as pd
import requests
import numpy as np
import requests
from bs4 import BeautifulSoup
import re



In [2]:
num_list = [1,2,3,4,5]
num_list.remove(2)
print(num_list)

[1, 3, 4, 5]


In [24]:
sal_data = pd.read_csv('sal_urls.csv')
sal_data.columns = ['index', 'url']

In [25]:
sal_data 

,index,url
0,10871,https://www.google.com/aclk?adurl=&ai=DChcSEwj...
1,10872,https://www.google.com/shopping/product/166466...
2,10873,https://www.google.com/shopping/product/777811...
3,10874,https://www.google.com/aclk?adurl=&ai=DChcSEwi...
4,10875,https://www.google.com/shopping/product/777811...
...,...,...
10866,21737,NaN
10867,21738,NaN
10868,21739,NaN
10869,21740,NaN


In [26]:
sal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10871 entries, 0 to 10870
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   10871 non-null  int64 
 1   url     9133 non-null   object
dtypes: int64(1), object(1)
memory usage: 170.0+ KB


In [7]:
def get_pdp(URL):
    try:
        if any(x in URL for x in ['google','wikipedia','bing','youtube','facebook','twitter','yahoo']):
            class_pdp = "search_page"
        elif len(URL) < 50:
            class_pdp = "skipped"
        else:
            HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
            webpage = requests.get(URL, headers=HEADERS, timeout=20)
            soup = BeautifulSoup(webpage.content, "html5lib")
            text = soup.find_all(text=True)
            new = []
            for i in text:
                new.append(i.lower())
            pdp = [i for i in new if "add to cart" in i]
            if len(pdp) == 0 or len(pdp)> 10:
                class_pdp = "NO"
            else:
                class_pdp = "YES"
    except:
        class_pdp = "timeout"

    return class_pdp


In [8]:
for i in sal_data.url:
    class_pdp = get_pdp(i)
    f = open('class_pdp.txt', 'a+')
    f.write(class_pdp+"\n")
    f.close()

In [17]:
f2 = open("class_pdp.txt", "r")
pdp_tag = f2.readlines()

pdp=[]
for i in pdp_tag:
    pdp.append(i[:-1])

In [29]:
pdp_df = pd.DataFrame(pdp, columns=['pdp'])
# add and index column starting from 10871
pdp_df['index'] = sal_data['index']
len_pfp = len(pdp_df)

In [30]:
len_pfp

4184

In [31]:
pdp_df

,pdp,index
0,search_page,10871
1,search_page,10872
2,search_page,10873
3,search_page,10874
4,search_page,10875
...,...,...
4179,NO,15050
4180,NO,15051
4181,search_page,15052
4182,search_page,15053


In [32]:
merged_df = pd.merge(sal_data, pdp_df, on='index')

In [33]:
merged_df

,index,url,pdp
0,10871,https://www.google.com/aclk?adurl=&ai=DChcSEwj...,search_page
1,10872,https://www.google.com/shopping/product/166466...,search_page
2,10873,https://www.google.com/shopping/product/777811...,search_page
3,10874,https://www.google.com/aclk?adurl=&ai=DChcSEwi...,search_page
4,10875,https://www.google.com/shopping/product/777811...,search_page
...,...,...,...
4179,15050,https://biccamera.rakuten.co.jp/item/871010384...,NO
4180,15051,https://review.rakuten.co.jp/item/1/269553_125...,NO
4181,15052,https://www.google.com/search?aqs=chrome..69i5...,search_page
4182,15053,https://www.youtube.com/watch?v=ZGD6t2929-k,search_page


In [39]:
df = pd.read_csv('ClickStream+PurchasesCombined_FINAL_15Feb (1).csv')
# Creat a new column for index
df['index'] = df.index

In [40]:
df.columns

Index(['Name', 'Total time spent on user test', 'Average time taken on tasks',
       'Average page views per task', 'User agent', 'Custom ID',
       'Date completed user test', 'Item', 'Task/Question', 'Page views',
       'Path Analysis', 'Navigation', 'Time spent', 'Task result', 'domain',
       'UUID', 'surveyID', 'TouchPoint', 'Sequence', 'Reverse Sequence',
       'Sponsored', 'Sponsored link', 'Domain_Type', 'DTC_Page',
       'Domain_Type2', 'SearchType', 'SearchType2', 'Gender', 'Country',
       'Survey', 'FullName', 'JourneyStage', 'ProdName', 'ratings', 'OID',
       'resid', 'User Id', 'searchTerm', 'PDP Flag', 'Prev_PDP', 'Prev_Brand',
       'Brand', 'URL_Cat', 'index'],
      dtype='object')

In [43]:
df = df[['Path Analysis','index','UUID',  'PDP Flag', 'URL_Cat']]

In [45]:
df_ckeck = pd.merge(df, merged_df, on='index')

In [47]:
df_ckeck.to_csv('df_ckeck.csv')